In [1]:
import itertools
from collections import OrderedDict
import numpy as np
import pickle
import analysis_constants as ac

In [2]:
label_map = {
    'input_x_gradients': 'Input X Gradient',
    'deeplift': 'DeepLIFT',
    'kernel_shap': 'Kernel SHAP',
    'integrated_gradients': 'Integrated Gradients',
    'lime': 'LIME',
    'guided_backprop': 'Guided Backprop'
}

In [3]:
all_term_dict = ac.load_all_calculated_terms_for_tasks()

In [4]:
def generate_alpha_combos():
    #returns alpha combinations in integers (0-10), because floating points aren't precise
    alphas = np.arange(0, 11, step=1, dtype=np.uint8)
    #also only need to generate combinations  of a1 and a2's because a3 = 1 - a1 - a2
    all_combos = [(a1,a2) for (a1,a2) in itertools.product(alphas, alphas) if np.sum([a1,a2]) <= 10]
#     all_combos.reverse()
    all_combos = sorted(all_combos, key=lambda x: (x[0],x[1]))
    return all_combos

In [5]:
alpha_combos = generate_alpha_combos()

In [6]:
#apply the combination of alphas to generate scores
#return a dictionary that is indexed by tuples of the form (a1, a2) 
def apply_alphas(m1, m2, m3, alpha_combos):
    score_dict = {} #indexed by the alpha combinations
    for combo in alpha_combos:
        a1, a2 = combo[0]/10.0, combo[1]/10.0 #divide by 10 to get floats. 
        a3 = 1 - (a1 + a2)
        a1m1, a2m2, a3m3 = a1 * m1, a2 * m2, a3 * m3
        score = a1m1 + a2m2 + a3m3
        score_dict[combo] = score
    return score_dict

In [7]:
def gen_scores_with_alpha_combos(all_term_dict, alpha_combos):
    term1_name, term2_name, term3_name = ac.term_names[0], ac.term_names[1], ac.term_names[2]
    all_scores_dict = {}
    #organize scores by tasks
    for task_name in ac.task_names:
        all_scores_dict[task_name] = {}
        #then by framework name
        for frame_name in ac.frame_names:
            t1, t2, t3 = all_term_dict[term1_name][task_name][frame_name],\
                         all_term_dict[term2_name][task_name][frame_name],\
                         all_term_dict[term3_name][task_name][frame_name]
            score_dict = apply_alphas(t1, t2, t3, alpha_combos)
            all_scores_dict[task_name][frame_name] = score_dict
        
        #for each task, sort the frames by average score
#         all_scores_dict[task_name] = OrderedDict(sorted(all_scores_dict[task_name].items(), key=lambda x:np.mean(x[1]), reverse=True))
        
        
    with open(f'{ac.IQS_alphas_graph_file}.pkl', 'wb') as f:
        pickle.dump(all_scores_dict, f)
        
    return all_scores_dict

In [8]:
#apply the combination of alphas to generate scores
#return a dictionary that is indexed by tuples of the form (a1, a2) 
def apply_alphas_13(m1, m2, m3, alpha_combos):
    score_dict = {} #indexed by the alpha combinations
    for combo in alpha_combos:
        a1, a2 = combo[0]/10.0, combo[1]/10.0 #divide by 10 to get floats. 
        a3 = 1 - (a1 + a2)
        a1m1, a2m2, a3m3 = a1 * m1, a2 * m2, a3 * m3
        score = a1m1 + a2m2 + a3m3
        score_dict[combo] = (a1m1, a2m2, a3m3, score)
    return score_dict

def gen_scores_with_alpha_combos_13(all_term_dict, alpha_combos):
    term1_name, term2_name, term3_name = ac.term_names[0], ac.term_names[1], ac.term_names[2]
    all_scores_dict = {}
    #organize scores by tasks
    for task_name in ac.task_names:
        all_scores_dict[task_name] = {}
        #then by framework name
        for frame_name in ac.frame_names:
            t1, t2, t3 = all_term_dict[term1_name][task_name][frame_name],\
                         all_term_dict[term2_name][task_name][frame_name],\
                         all_term_dict[term3_name][task_name][frame_name]
            score_dict = apply_alphas_13(t1, t2, t3, alpha_combos)
            all_scores_dict[task_name][frame_name] = score_dict
        
        #for each task, sort the frames by average score
#         all_scores_dict[task_name] = OrderedDict(sorted(all_scores_dict[task_name].items(), key=lambda x:np.mean(x[1]), reverse=True))
        
    return all_scores_dict

In [9]:
all_scores_dict = gen_scores_with_alpha_combos(all_term_dict, alpha_combos)

In [10]:
all_scores_dict

{'sst2': {'input_x_gradients': {(0, 0): 0.7466682052950411,
   (0, 1): 0.7440980036570367,
   (0, 2): 0.7415278020190323,
   (0, 3): 0.7389576003810279,
   (0, 4): 0.7363873987430236,
   (0, 5): 0.7338171971050191,
   (0, 6): 0.7312469954670149,
   (0, 7): 0.7286767938290105,
   (0, 8): 0.726106592191006,
   (0, 9): 0.7235363905530017,
   (0, 10): 0.7209661889149973,
   (1, 0): 0.7385320296295151,
   (1, 1): 0.7359618279915108,
   (1, 2): 0.7333916263535063,
   (1, 3): 0.730821424715502,
   (1, 4): 0.7282512230774977,
   (1, 5): 0.7256810214394933,
   (1, 6): 0.723110819801489,
   (1, 7): 0.7205406181634846,
   (1, 8): 0.7179704165254802,
   (1, 9): 0.7154002148874757,
   (2, 0): 0.7303958539639893,
   (2, 1): 0.7278256523259847,
   (2, 2): 0.7252554506879805,
   (2, 3): 0.7226852490499761,
   (2, 4): 0.7201150474119717,
   (2, 5): 0.7175448457739674,
   (2, 6): 0.7149746441359629,
   (2, 7): 0.7124044424979586,
   (2, 8): 0.7098342408599543,
   (3, 0): 0.7222596782984633,
   (3, 1): 0

In [11]:
frame_avg_dict = {}
for frame_name in ac.frame_names:
    line_str = ''
    frame_sum = []
    line_str += f'\\textbf{{{label_map[frame_name]}}}'
    for task_name in ac.task_names:
        task_sum = []
        for term_name in ac.term_names:
            score = all_term_dict[term_name][task_name][frame_name]
            frame_sum.append(score)
            task_sum.append(score)
        line_str += f'& {np.mean(task_sum):.4f} \\tiny ({np.std(task_sum):.4f})'   
    line_str += f'& {0.0000:.4f} \\tiny ({0.0000:.4f}) & {0.0000:.4f} \\tiny ({0.0000:.4f}) & {0.0000:.4f} \\tiny ({0.0000:.4f})'
    line_str += '\\\\\n'
    print(line_str)
    frame_avg_dict[frame_name] = np.mean(frame_sum)
print(frame_avg_dict)

\textbf{Input X Gradient}& 0.7110 \tiny (0.0340)& 0.6695 \tiny (0.2056)& 0.5747 \tiny (0.2366)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

\textbf{DeepLIFT}& 0.7106 \tiny (0.0403)& 0.6678 \tiny (0.2014)& 0.5736 \tiny (0.2263)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

\textbf{Kernel SHAP}& 0.7075 \tiny (0.0194)& 0.6235 \tiny (0.2492)& 0.5804 \tiny (0.2138)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

\textbf{LIME}& 0.5962 \tiny (0.1377)& 0.6098 \tiny (0.2684)& 0.5520 \tiny (0.2287)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

\textbf{Guided Backprop}& 0.5831 \tiny (0.1283)& 0.6122 \tiny (0.2640)& 0.5754 \tiny (0.2334)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

\textbf{Integrated Gradients}& 0.5920 \tiny (0.1179)& 0.5173 \tiny (0.3014)& 0.5527 \tiny (0.1810)& 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000) & 0.0000 \tiny (0.0000)\\

{'input_

In [12]:
for frame_name in ac.frame_names:
    frame_sum = []
    for task_name in ac.task_names:
        task_sum = []
        for term_name in ac.term_names:
            score = all_term_dict[term_name][task_name][frame_name]
            frame_sum.append(score)
            task_sum.append(score)
        print(f'{task_name} avg: {np.mean(task_sum):.4f} std: {np.std(task_sum):.4f}')
    print(f'{frame_name} avg: {np.mean(frame_sum):.4f} std: {np.std(frame_sum):.4f}')

sst2 avg: 0.7110 std: 0.0340
stsb avg: 0.6695 std: 0.2056
qnli avg: 0.5747 std: 0.2366
input_x_gradients avg: 0.6517 std: 0.1908
sst2 avg: 0.7106 std: 0.0403
stsb avg: 0.6678 std: 0.2014
qnli avg: 0.5736 std: 0.2263
deeplift avg: 0.6507 std: 0.1855
sst2 avg: 0.7075 std: 0.0194
stsb avg: 0.6235 std: 0.2492
qnli avg: 0.5804 std: 0.2138
kernel_shap avg: 0.6371 std: 0.1971
sst2 avg: 0.5962 std: 0.1377
stsb avg: 0.6098 std: 0.2684
qnli avg: 0.5520 std: 0.2287
lime avg: 0.5860 std: 0.2199
sst2 avg: 0.5831 std: 0.1283
stsb avg: 0.6122 std: 0.2640
qnli avg: 0.5754 std: 0.2334
guided_backprop avg: 0.5902 std: 0.2171
sst2 avg: 0.5920 std: 0.1179
stsb avg: 0.5173 std: 0.3014
qnli avg: 0.5527 std: 0.1810
integrated_gradients avg: 0.5540 std: 0.2163


In [13]:
onethird = gen_scores_with_alpha_combos_13(all_term_dict, [(10/3, 10/3)])

In [14]:
for task_name in ac.task_names:
    print(f'task name: {task_name}')
    task_scores = onethird[task_name]
    for frame_name in ac.frame_names:
        frame_scores = task_scores[frame_name]#[1]
        for onethird_combo, one_third_scores in frame_scores.items():
            print(f'\\textbf{{{label_map[frame_name]}}}  & {one_third_scores[0]:.4f} & {one_third_scores[1]:.4f} & {one_third_scores[2]:.4f} & {one_third_scores[3]:.4f} \\\\ ')
        
    

task name: sst2
\textbf{Input X Gradient}  & 0.2218 & 0.2403 & 0.2489 & 0.7110 \\ 
\textbf{DeepLIFT}  & 0.2189 & 0.2403 & 0.2513 & 0.7106 \\ 
\textbf{Kernel SHAP}  & 0.2302 & 0.2324 & 0.2449 & 0.7075 \\ 
\textbf{LIME}  & 0.1555 & 0.1784 & 0.2623 & 0.5962 \\ 
\textbf{Guided Backprop}  & 0.1607 & 0.1677 & 0.2547 & 0.5831 \\ 
\textbf{Integrated Gradients}  & 0.1530 & 0.1905 & 0.2485 & 0.5920 \\ 
task name: stsb
\textbf{Input X Gradient}  & 0.2195 & 0.1411 & 0.3089 & 0.6695 \\ 
\textbf{DeepLIFT}  & 0.2212 & 0.1411 & 0.3056 & 0.6678 \\ 
\textbf{Kernel SHAP}  & 0.2155 & 0.1025 & 0.3056 & 0.6235 \\ 
\textbf{LIME}  & 0.1899 & 0.1010 & 0.3189 & 0.6098 \\ 
\textbf{Guided Backprop}  & 0.2011 & 0.0978 & 0.3133 & 0.6122 \\ 
\textbf{Integrated Gradients}  & 0.1177 & 0.0863 & 0.3133 & 0.5173 \\ 
task name: qnli
\textbf{Input X Gradient}  & 0.2750 & 0.0857 & 0.2140 & 0.5747 \\ 
\textbf{DeepLIFT}  & 0.2580 & 0.0857 & 0.2298 & 0.5736 \\ 
\textbf{Kernel SHAP}  & 0.2566 & 0.0939 & 0.2300 & 0.5804 \\ 
\tex

In [15]:
onethird.keys()

dict_keys(['sst2', 'stsb', 'qnli'])

In [16]:
for frame_name in ac.frame_names:
    frame_sum = []
    for term_name in ac.term_names:
        term_sum = []
        for task_name in ac.task_names:
            score = all_term_dict[term_name][task_name][frame_name]
            term_sum.append(score)
            frame_sum.append(score)
        print(f'{term_name} avg: {np.mean(term_sum):.4f} std: {np.std(term_sum):.4f}')
    print(f'{frame_name} avg: {np.mean(frame_sum):.4f} std: {np.std(frame_sum):.4f}')
    print(f'=============================================')

plausibility avg: 0.7162 std: 0.0769
simplicity avg: 0.4672 std: 0.1919
reproducibility avg: 0.7718 std: 0.1176
input_x_gradients avg: 0.6517 std: 0.1908
plausibility avg: 0.6981 std: 0.0537
simplicity avg: 0.4672 std: 0.1919
reproducibility avg: 0.7867 std: 0.0956
deeplift avg: 0.6507 std: 0.1855
plausibility avg: 0.7022 std: 0.0510
simplicity avg: 0.4288 std: 0.1902
reproducibility avg: 0.7804 std: 0.0981
kernel_shap avg: 0.6371 std: 0.1971
plausibility avg: 0.6007 std: 0.1243
simplicity avg: 0.3577 std: 0.1285
reproducibility avg: 0.7995 std: 0.1235
lime avg: 0.5860 std: 0.2199
plausibility avg: 0.6220 std: 0.1225
simplicity avg: 0.3484 std: 0.1108
reproducibility avg: 0.8003 std: 0.1025
guided_backprop avg: 0.5902 std: 0.2171
plausibility avg: 0.5088 std: 0.1517
simplicity avg: 0.3768 std: 0.1387
reproducibility avg: 0.7764 std: 0.1229
integrated_gradients avg: 0.5540 std: 0.2163


In [17]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [18]:
kf_col_iqs = ['SST2'] #, 'STSB', 'QNLI', 'MRPC', 'CNN', 'SQuAD']
kf_col_ratings = [k+'_rating' for k in kf_col_iqs]
df = pd.DataFrame(columns = kf_col_iqs + kf_col_ratings)


In [19]:
df['SST2'] = [0.7218, 0.6845, 0.4756, 0.3512, 0.2312]
df['SST2_rating'] = [0.7134, 0.6987, 0.4532, 0.3146, 0.1134]

In [20]:
calculate_pvalues(df)

,SST2,SST2_rating
SST2,0.0,0.0004
SST2_rating,0.0004,0.0


In [21]:
pearsonr(df['SST2'], df['SST2_rating'])

(0.9954766522507723, 0.0003649462740265229)

In [22]:
df['STSB'] = [0.6218, 0.5845, 0.4256, 0.3112, 0.1312]
df['STSB_rating'] = [0.613, 0.4931, 0.4111, 0.3000, 0.074]
pearsonr(df['STSB'], df['STSB_rating'])

(0.9840042604177075, 0.0024226772836892395)